# Agentic Customer Chatbot

Here we are going to be testing both a Crewai approach and Llama-index ContextAugmentaion approach with RAG equivalent.

## Crewai Approach

Defining the Agents:

- Order Information Agent: Provides order status, estimated delivery dates, and order history.
- Product Information Agent: Supplies detailed information about products, categories, and availability.
- Review Analysis Agent: Summarizes and analyzes customer reviews to provide insights on product quality and customer satisfaction.


In [ ]:
from crewai import Agent, Task, Crew

# Order Information Agent
order_info_agent = Agent(
    role='Order Information Specialist',
    goal='Provide accurate order status and delivery information.',
    backstory="You have extensive experience in logistics and customer service.",
    tools=[]  # Specify any tools like API access to order database
)

# Product Information Agent
product_info_agent = Agent(
    role='Product Information Specialist',
    goal='Provide detailed information about products and their availability.',
    backstory="You are well-versed in product specifications and inventory management.",
    tools=[]  # Specify any tools like product database access
)

# Review Analysis Agent
review_analysis_agent = Agent(
    role='Review Analyst',
    goal='Analyze customer reviews to provide insights on product quality.',
    backstory="You have a background in data analysis and customer satisfaction.",
    tools=[]  # Specify any tools for sentiment analysis or text summarization
)


## Creating Tasks for Each Agent

Next, as per the Crewai approach, we will create tasks for each agent. The tasks are as follows:

In [ ]:
# Task for Order Information Agent
task_order_info = Task(
    description="Fetch order status, estimated delivery dates, and order history.",
    expected_output="Order status, delivery dates, and order history details.",
    agent=order_info_agent
)

# Task for Product Information Agent
task_product_info = Task(
    description="Provide detailed information about products including categories and availability.",
    expected_output="Detailed product descriptions and availability status.",
    agent=product_info_agent
)

# Task for Review Analysis Agent
task_review_analysis = Task(
    description="Summarize and analyze customer reviews to offer insights on product quality.",
    expected_output="Summary of reviews and insights on product quality.",
    agent=review_analysis_agent
)


## Combining Agents and Tasks into a Crew:

Next, we will combine the agents and tasks into a Crew. The Crew will be responsible for handling all customer queries and providing the necessary information.


In [ ]:
# Define the Crew
customer_support_crew = Crew(
    agents=[order_info_agent, product_info_agent, review_analysis_agent],
    tasks=[task_order_info, task_product_info, task_review_analysis],
    verbose=True
)

# Start the Crew
results = customer_support_crew.kickoff()
print(results)


### Potential Customization and Expansion:

- Add Tools: Integrate specific tools for each agent, such as databases, APIs, or sentiment analysis tools.
- Local and Cloud Models: Mix and match different models based on the requirements (e.g., using OpenAI models for natural language processing tasks).


# Agents using LLama-index

So we had a Uber Fillings example - where we had learnt about context augmentation agents. 
We are considering those as well to compare and contrast against Crewai agents.

## TODO
Create or Load Indexes:

We would need prepare our customer support documents and build or load indexes similarly to the Uber example.

Create Agents and Tools:

Next, we will Define agents for different customer service tasks: Order Information, Product Information, and Review Analysis.


In [ ]:
import json
from typing import Sequence
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai_legacy import ContextRetrieverOpenAIAgent
from llama_index.core import Document

In [ ]:
order_docs = SimpleDirectoryReader(input_files=["path_to_order_documents"]).load_data()
product_docs = SimpleDirectoryReader(input_files=["path_to_product_documents"]).load_data()
review_docs = SimpleDirectoryReader(input_files=["path_to_review_documents"]).load_data()

order_index = VectorStoreIndex.from_documents(order_docs)
product_index = VectorStoreIndex.from_documents(product_docs)
review_index = VectorStoreIndex.from_documents(review_docs)

order_engine = order_index.as_query_engine(similarity_top_k=3)
product_engine = product_index.as_query_engine(similarity_top_k=3)
review_engine = review_index.as_query_engine(similarity_top_k=3)

order_tool = QueryEngineTool(
    query_engine=order_engine,
    metadata=ToolMetadata(
        name="order_info",
        description="Provides information about order status and history."
    ),
)

product_tool = QueryEngineTool(
    query_engine=product_engine,
    metadata=ToolMetadata(
        name="product_info",
        description="Provides detailed product information and availability."
    ),
)

review_tool = QueryEngineTool(
    query_engine=review_engine,
    metadata=ToolMetadata(
        name="review_analysis",
        description="Analyzes customer reviews to provide product insights."
    ),
)


We are working on this part and will update this notebook soon.

### Instantiate the Context-Augmented Agent:

Once done, we will use a context index to provide additional context for the agent's decisions.


In [ ]:
from llama_index.core import Document
from llama_index.agent.openai_legacy import ContextRetrieverOpenAIAgent

context_texts = [
    "FAQ: What is the return policy?",
    "FAQ: How to track my order?",
    "FAQ: What are the shipping options?"
]
context_docs = [Document(text=t) for t in context_texts]
context_index = VectorStoreIndex.from_documents(context_docs)

context_agent = ContextRetrieverOpenAIAgent.from_tools_and_retriever(
    [order_tool, product_tool, review_tool],
    context_index.as_retriever(similarity_top_k=1),
    verbose=True,
)

response = context_agent.chat("How can I track my order?")
print(response)
